In [23]:
import sqlite3
from konk_api import query

ordbank = {
    "nob": "/disk1/bokhylla/ordbank.db",
    "nno" : "/disk1/bokhylla/ordbank_nno.db"
}


def get_paradigms(word, lang = 'nob'):
    res = query(ordbank[lang],"""
        select 
            substr(a.tag, 1, 4) as cat, 
            group_concat(distinct a.oppslag) 
        from 
            fullformsliste as a, 
            fullformsliste as b
            
        where 
            b.oppslag = ? 
            and b.lemma_id=a.lemma_id 
            
        group by
            cat""", (word,))
    result = []
    if res != []:
        for r in res:
            result.append([r[0].strip(), r[1].split(',')])
    return result

def get_paradigms_many(words, lang = 'nob'):
    with sqlite3.connect(ordbank[lang]) as con:
        cur = con.cursor()
        cur.execute("attach '' as words")
        cur.execute("create table words.wordlist (word varchar)")
        for word in words:
            cur.execute("insert into words.wordlist values (?)", (word,))
        res = cur.execute("""
            select 
                wl.word,
                substr(a.tag, 1, 4) as cat, 
                group_concat(distinct a.oppslag) 
            from 
                fullformsliste as a, 
                fullformsliste as b,
                words.wordlist as wl
            where 
                b.oppslag = wl.word
                and b.lemma_id=a.lemma_id 

            group by
                wl.word, cat""").fetchall()
        result = []
        if res != []:
            for r in res:
                result.append([r[0], r[1].strip(), r[2].split(',')])
    return result

def get_lemma(word, lang = 'nob'):
    res = query(ordbank[lang],"""
        select 
            distinct
            substr(a.tag, 1, 4) as cat, 
            l.GRUNNFORM
        from 
            fullformsliste as a, 
            lemma as l
        where 
            a.oppslag = ? 
            and l.lemma_id = a.lemma_id
        """, (word,))
    result = []
    if res != []:
        for r in res:
            result.append([r[0].strip(), r[1]])
    return result


def get_lemma_many(words, lang = 'nob'):
    with sqlite3.connect(ordbank[lang]) as con:
        cur = con.cursor()
        cur.execute("attach '' as words")
        cur.execute("create table words.wordlist (word varchar)")
        for word in words:
            cur.execute("insert into words.wordlist values (?)", (word,))
        res = cur.execute("""
        select 
            distinct
            wl.word,
            substr(a.tag, 1, 4) as cat, 
            l.GRUNNFORM
        from 
            fullformsliste as a, 
            lemma as l,
            words.wordlist as wl
        where 
            a.oppslag = wl.word
            and l.lemma_id = a.lemma_id
        """).fetchall()
    result = []
    if res != []:
        for r in res:
            result.append([r[0], r[1].strip(), r[2]])
    return result

def get_form(word, lang = 'nob'):
    res = query(ordbank[lang],"""
        select distinct
            tag
        from 
            fullformsliste 
        where 
            oppslag=?""", (word,))
    result = []
    if res != []:
        for r in res:
            result.append(r[0].strip())
    return result

def get_form_many(words, lang = 'nob'):
    with sqlite3.connect(ordbank[lang]) as con:
        cur = con.cursor()
        cur.execute("attach '' as words")
        cur.execute("create table words.wordlist (word varchar)")
        for word in words:
            cur.execute("insert into words.wordlist values (?)", (word,))
        res = cur.execute("""
            select distinct
                wl.word,
                f.tag
            from 
                fullformsliste as f,
                words.wordlist as wl
            where 
                f.oppslag=wl.word""")
        result = []
        if res != []:
            for r in res:
                result.append([r[0], r[1].strip()])
    return result

def get_variant(word, gramtype, lang = 'nob'):
    res = query(ordbank[lang],"""
        select distinct
            a.tag as cat, 
            a.oppslag 
        from 
            fullformsliste as a, 
            fullformsliste as b 
        where 
            b.oppslag = ? 
            and b.lemma_id=a.lemma_id
            and a.tag like ?
        group by
            cat""", (word, '%' + gramtype + '%'))
    result = []
    if res != []:
        for r in res:
            result.append((r[0].strip(), r[1]))
    return result




In [25]:
get_form_many(['spisande','sovande'], lang = 'nno')

[['sovande',
  'adj <pres-part> <intrans2> <part4/hen> <part4/inn> <part4/over> <part4/ut> <part1/bort> <trans1> normert']]

In [27]:
get_lemma_many(['etande', 'leande'], lang = 'nno')

[['etande', 'subs', 'etande'],
 ['etande', 'adj', 'etande'],
 ['etande', 'adj', 'ete'],
 ['leande', 'adj', 'le'],
 ['leande', 'adj', 'lee']]

In [29]:
get_paradigms('fiskane', lang='nno')

[['subs', ['fiskar', 'fisken', 'fisk', 'fiskane']]]

In [49]:
get_paradigms_many(['eter', 'sover', 'leande'], lang='nno')

[['eter',
  'subs',
  ['ete', 'etene', 'eter', 'eta', 'eterane', 'eterar', 'eteren']],
 ['leande', 'adj', ['ledde', 'ledd', 'leande', 'lett', 'lea']],
 ['leande',
  'verb',
  ['le', 'ler', 'least', 'lo', 'ledd', 'lett', 'lee', 'lea', 'lear']]]

In [65]:
get_form_many(['tørst'], lang='nno')

[['tørst', 'verb pres <st-verb> normert'],
 ['tørst', 'adj pos m/f ub eint normert'],
 ['tørst', 'adj pos nøyt ub eint normert'],
 ['tørst', 'adj <perf-part> nøyt ub eint <intrans3> <trans15/etter> normert'],
 ['tørst', 'adj <perf-part> m/f ub eint <intrans3> <trans15/etter> normert'],
 ['tørst', 'verb imp <intrans3> <trans15/etter> normert'],
 ['tørst', 'verb perf-part <intrans3> <trans15/etter> normert']]

In [64]:
get_variant('tørst', 'pret', lang='nob')

[('verb pret <intrans3> <trans15/etter> normert', 'tørstet')]

In [7]:
query(ordbank, "select * from sqlite_master")

[('table',
  'fullformsliste',
  'fullformsliste',
  2,
  'CREATE TABLE "fullformsliste" ("loepenr" INTEGER, "LEMMA_ID" INTEGER, "OPPSLAG" VARCHAR, "TAG" VARCHAR, "PARADIGME_ID" INTEGER, "BOY_NUMMER" INTEGER)'),
 ('table',
  'lemma',
  'lemma',
  5,
  'CREATE TABLE "lemma" ("loepenr" INTEGER, "LEMMA_ID" INTEGER, "GRUNNFORM" VARCHAR, "BM_ORDBOK" VARCHAR)'),
 ('index',
  'ff_lemma',
  'fullformsliste',
  2915,
  'CREATE INDEX "ff_lemma" ON "fullformsliste" ("LEMMA_ID" ASC)'),
 ('index',
  'ff_oppslag',
  'fullformsliste',
  3169,
  'CREATE INDEX "ff_oppslag" ON "fullformsliste" ("OPPSLAG" ASC)'),
 ('index',
  'ff_tag',
  'fullformsliste',
  3581,
  'CREATE INDEX "ff_tag" ON "fullformsliste" ("TAG" ASC)'),
 ('index',
  'lma_gform',
  'lemma',
  4523,
  'CREATE INDEX "lma_gform" ON "lemma" ("GRUNNFORM" ASC)'),
 ('index',
  'lma_id',
  'lemma',
  4568,
  'CREATE INDEX "lma_id" ON "lemma" ("LEMMA_ID" ASC)'),
 ('view',
  'ord',
  'ord',
  0,
  'CREATE VIEW ord as select a.oppslag, b.grunnform

In [8]:
query(ordbank, "select * from fullformsliste where oppslag = 'glimrende'")

[(194116, 23683, 'glimrende', 'adj <pres-part> <intrans2> normert', 1, 10),
 (194117, 23683, 'glimrende', 'adj <pres-part> <intrans2> normert', 7, 10),
 (194118, 23683, 'glimrende', 'adj <pres-part> <intrans2> normert', 10, 10),
 (194119, 23683, 'glimrende', 'adj <pres-part> <intrans2> normert', 11, 10),
 (194120, 23683, 'glimrende', 'adj <pres-part> <intrans2> normert', 17, 10),
 (194121, 23683, 'glimrende', 'adj <pres-part> <intrans2> normert', 19, 10),
 (194154, 23684, 'glimrende', 'adj pos be ent normert', 513, 3),
 (194155, 23684, 'glimrende', 'adj pos fl normert', 513, 2),
 (194156, 23684, 'glimrende', 'adj pos m/f ub ent normert', 513, 1),
 (194157, 23684, 'glimrende', 'adj pos nøyt ub ent normert', 513, 4)]

In [13]:
query(ordbank, "select * from lemma where lemma_id = 23684")

[(21747, 23684, 'glimrende', 'BM_ORDBOK')]

In [ ]:
query()

In [39]:
q = lambda x, y=(): query(ordbank, x, param=y, size=0)

In [122]:
q("select distinct g.oppslag, g.tag from fullformsliste as f, fullformsliste as g, lemma as l where f.oppslag ='mannens' and g.lemma_id = f.lemma_id")

[]

In [49]:
q("select substr(a.tag, 1,4), group_concat(distinct a.oppslag) from fullformsliste as a, fullformsliste as b where b.oppslag = 'egget' and b.lemma_id=a.lemma_id group by b.lemma_id")

[(13720, 'subs', 'egg,egga,eggene,egget'),
 (13725, 'adj ', 'egg,egga,egge,eggede,eggende,egger,egges,egget,eggete')]

In [126]:
q("select * from lemma where lemma = 'mannens'")

en feil oppstod: no such column: lemma


[]

In [128]:
q("select * from sqlite_master")

[('table',
  'fullformsliste',
  'fullformsliste',
  2,
  'CREATE TABLE "fullformsliste" ("loepenr" INTEGER, "LEMMA_ID" INTEGER, "OPPSLAG" VARCHAR, "TAG" VARCHAR, "PARADIGME_ID" INTEGER, "BOY_NUMMER" INTEGER)'),
 ('table',
  'lemma',
  'lemma',
  5,
  'CREATE TABLE "lemma" ("loepenr" INTEGER, "LEMMA_ID" INTEGER, "GRUNNFORM" VARCHAR, "BM_ORDBOK" VARCHAR)'),
 ('index',
  'ff_lemma',
  'fullformsliste',
  2915,
  'CREATE INDEX "ff_lemma" ON "fullformsliste" ("LEMMA_ID" ASC)'),
 ('index',
  'ff_oppslag',
  'fullformsliste',
  3169,
  'CREATE INDEX "ff_oppslag" ON "fullformsliste" ("OPPSLAG" ASC)'),
 ('index',
  'ff_tag',
  'fullformsliste',
  3581,
  'CREATE INDEX "ff_tag" ON "fullformsliste" ("TAG" ASC)'),
 ('index',
  'lma_gform',
  'lemma',
  4523,
  'CREATE INDEX "lma_gform" ON "lemma" ("GRUNNFORM" ASC)'),
 ('index',
  'lma_id',
  'lemma',
  4568,
  'CREATE INDEX "lma_id" ON "lemma" ("LEMMA_ID" ASC)'),
 ('view',
  'ord',
  'ord',
  0,
  'CREATE VIEW ord as select a.oppslag, b.grunnform

In [66]:
get_lemma_many('skriver sover spiste trodde'.split())

[['skriver', 'verb', 'skrive'],
 ['skriver', 'subs', 'skriver'],
 ['sover', 'verb', 'sove'],
 ['spiste', 'adj', 'spise'],
 ['spiste', 'verb', 'spise'],
 ['trodde', 'adj', 'tro'],
 ['trodde', 'verb', 'tro']]

In [68]:
onob = ordbank['nob']
onno = ordbank['nno']

In [73]:
query(onno, "select * from sqlite_master")

[('table',
  'fullformsliste',
  'fullformsliste',
  297,
  'CREATE TABLE fullformsliste(\n  "LOEPENR" TEXT,\n  "LEMMA_ID" TEXT,\n  "OPPSLAG" TEXT,\n  "TAG" TEXT,\n  "PARADIGME_ID" TEXT,\n  "BOY_NUMMER" TEXT,\n  "FRADATO" TEXT,\n  "TILDATO" TEXT,\n  "NORMERING" TEXT\n)'),
 ('table',
  'lemma',
  'lemma',
  2333,
  'CREATE TABLE lemma(\n  "LOEPENR" TEXT,\n  "LEMMA_ID" TEXT,\n  "GRUNNFORM" TEXT,\n  "\'NOB_2012\'" TEXT\n)'),
 ('index',
  'ff_lemma',
  'fullformsliste',
  1876,
  'CREATE INDEX "ff_lemma" ON "fullformsliste" ("LEMMA_ID" ASC)'),
 ('index',
  'ff_oppslag',
  'fullformsliste',
  24566,
  'CREATE INDEX "ff_oppslag" ON "fullformsliste" ("OPPSLAG" ASC)'),
 ('index',
  'ff_tag',
  'fullformsliste',
  16684,
  'CREATE INDEX "ff_tag" ON "fullformsliste" ("TAG" ASC)'),
 ('index',
  'lma_gform',
  'lemma',
  15657,
  'CREATE INDEX "lma_gform" ON "lemma" ("GRUNNFORM" ASC)'),
 ('index',
  'lma_id',
  'lemma',
  16603,
  'CREATE INDEX "lma_id" ON "lemma" ("LEMMA_ID" ASC)')]

In [ ]:
with sqlite3.connect(onob) as con:
    cur = con.cursor()
    cur.execute("attach '{onno}' as nno".format(onno=onno))
    cur.execute("select count(*) from fullformsliste as b, nno.fullformsliste as n where b.oppslag = n.oppslag ")
    res = cur.fetchall()
res